# Tensorflow Demo
playing with the [Deep ConvNet for MNIST digit classification](https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#build-a-multilayer-convolutional-network) tutorial

## Import MNIST data

In [5]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST-data', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


## simple softmax multinomial regression

In [16]:
import tensorflow as tf
sess = tf.InteractiveSession()

# set input/output placeholders
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])
# define the model parameters as tf.Variable's
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# initialize them in our InteractiveSession
sess.run(tf.initialize_all_variables())

y = tf.nn.softmax(tf.matmul(x,W) + b)

# Loss function = the discrete cross entropy between the predicted distributions and the labels 1-hot vectors
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# use steepest descent to minmize loss function, 
# train_step is an op that will compute gradient, compute update step, apply update
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# So to find min, just repeated run the train_step
for i in xrange(1000):
    if i % 100 == 0:
        print i
    batch = mnist.train.next_batch(50)
    # note, using feed_dict can replace any tensor in graph, not just tf.placeholders
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
# evaluate
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accu = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

print(accu.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

sess.close()

0
100
200
300
400
500
600
700
800
900
0.9089


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x1100e46d0>> ignored


In [11]:
import tensorflow as tf
import numpy as np

A = np.random.random([3,3])
print(A)
b = np.array([[1],[2],[3]]).astype(float)
sess = tf.Session()
print(sess.run(tf.constant(A)+tf.constant(b)))

[[ 0.18985606  0.85909412  0.90885811]
 [ 0.42509889  0.95442399  0.55010974]
 [ 0.53262118  0.22159028  0.77416818]]
[[ 1.18985606  1.85909412  1.90885811]
 [ 2.42509889  2.95442399  2.55010974]
 [ 3.53262118  3.22159028  3.77416818]]


## Deep ConvNet demo

In [1]:
import tensorflow as tf

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


hello tf
